# 잠재 디리클레 할당(LDA) 실습2


- 교재 6.3절에 있는 잠재디리클레 할당(LDA)실습 2의 실습내용을 수행한 코드 및 출력 결과입니다.
- 1. 교재의 6.3절에 있는 LDA 분석 프로그램을 수행하여 최종 결과를 얻는 문제의 답안입니다.

## 1. 데이터 읽어오기

- 링크 : https://www.kaggle.com/therohk/million-headlines
- 약 15년간 발행했던 뉴스 기사 제목을 모아둔 데이터

In [8]:
# 15년간 뉴스 기사 제목 데이터를 URL을 통해 받아옴
# 링크 :  https://www.kaggle.com/therohk/million-headlines
import pandas as pd 
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

print(len(data))

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


1082168


## 2.  데이터 내용

In [2]:
# 5개 확인
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [3]:
# 100개 확인
print(data.head(100))

    publish_date                                      headline_text
0       20030219  aba decides against community broadcasting lic...
1       20030219     act fire witnesses must be aware of defamation
2       20030219     a g calls for infrastructure protection summit
3       20030219           air nz staff in aust strike for pay rise
4       20030219      air nz strike to affect australian travellers
..           ...                                                ...
95      20030219           meeting to consider tick clearance costs
96      20030219         meeting to focus on broken hill water woes
97      20030219                      moderate lift in wages growth
98      20030219      more than 40 pc of young men drink alcohol at
99      20030219  more water restrictions predicted for northern...

[100 rows x 2 columns]


## 3. 텍스트 전처리

- 영어 단어에 대한 불용어 제거, 표제어 추출, 길이가 짧은 단어 제거 등을 수행
1. nltk의 word_tokenize를 실행하면 문장을 단어로 분리
2. nltk의 stopwords를 이용하여 불용어를 제거
3. nltk의 WordNetLemmatizer를 이용하여 동사 단어들을 정제(과거를 현재로, 3인칭을 1인칭으로)
4. 단어 길이가 3이하인 단어들(aba, act, nz, air 등)을 제거

### 3.1 텍스트 전처리 1

- 속도가 느리나, 이런 경우 Google Colab을 활용하면 빠르게 문제를 처리할 수 있음

In [4]:
# column이름만 추출
text = data[['headline_text']]

import nltk

# word tokenization
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# stop words removal 불용어 제거
from nltk.corpus import stopwords

stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])

print(text.head(5))

C:\Users\user\AppData\Local\Temp/ipykernel_6280/4206435398.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)


                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


C:\Users\user\AppData\Local\Temp/ipykernel_6280/4206435398.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])


### 3.2 텍스트 전처리 2

- 길이가 작은 단어들 처리 (길이가 3이하의 단어들 제거 )

In [5]:
# Lemmatization

from nltk.stem import WordNetLemmatizer

text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3]) 

print(tokenized_doc[:5])

C:\Users\user\AppData\Local\Temp/ipykernel_6280/2041771923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])


0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


## 4. tf-idf 행렬 만들기

- sklearn 함수인 TfidfVectorizer는 입력 데이터로 단어 집합이 아니라 문장을 사용함: 정제된 단어들에 대해 detokenize를 수행하여 문장을 다시 만듦

In [6]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

from sklearn.feature_extraction.text import TfidfVectorizer
# 상위 1,000개의 단어를 보존
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)

X = vectorizer.fit_transform(text['headline_text']) 
# TF-IDF 행렬의 크기 확인 
X.shape 

C:\Users\user\AppData\Local\Temp/ipykernel_6280/2844421375.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장


(1082168, 1000)

## 5. 토픽 모델링

In [7]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1) 
lda_top=lda_model.fit_transform(X)

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨. 

def get_topics(components, feature_names, n=5):
	for idx, topic in enumerate(components):
		print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('government', 8725.19), ('sydney', 8393.29), ('queensland', 7720.12), ('change', 5874.27), ('home', 5674.38)]
Topic 2: [('australia', 13691.08), ('australian', 11088.95), ('melbourne', 7528.43), ('world', 6707.7), ('south', 6677.03)]
Topic 3: [('death', 5935.06), ('interview', 5924.98), ('kill', 5851.6), ('jail', 4632.85), ('life', 4275.27)]
Topic 4: [('house', 6113.49), ('2016', 5488.19), ('state', 4923.41), ('brisbane', 4857.21), ('tasmania', 4610.97)]
Topic 5: [('court', 7542.74), ('attack', 6959.64), ('open', 5663.0), ('face', 5193.63), ('warn', 5115.01)]
Topic 6: [('market', 5545.86), ('rural', 5502.89), ('plan', 4828.71), ('indigenous', 4223.4), ('power', 3968.26)]
Topic 7: [('charge', 8428.8), ('election', 7561.63), ('adelaide', 6758.36), ('make', 5658.99), ('test', 5062.69)]
Topic 8: [('police', 12092.44), ('crash', 5281.14), ('drug', 4290.87), ('beat', 3257.58), ('rise', 2934.92)]
Topic 9: [('fund', 4693.03), ('labor', 4047.69), ('national', 4038.68), ('council', 40